In [1]:
# First we want to check what version of python we are running here: 
import sys
print("Python version: " + str(sys.version_info.major) + "." + str(sys.version_info.minor) + "." + str(sys.version_info.micro))

Python version: 3.8.3


In [2]:
!pip install torch
!pip install torchtext

In [1]:
import torchtext
print("Successfully imported TorchText module from PyTorch.")

Successfully imported TorchText module from PyTorch.


In [3]:
# Import some datasets from TorchText
from torchtext.datasets import text_classification
print("Successfully imported some datasets from TorchText.")

Successfully imported some datasets from TorchText.


In [4]:
# Creating a folder called 'data'
import os
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
    
# Specifying how to break up the dataset into n-words 
NGRAMS = 2

# Load the training dataset and testing dataset
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=NGRAMS, vocab=None)

.data\ag_news_csv.tar.gz: 11.8MB [00:02, 5.04MB/s]
120000lines [00:31, 3816.44lines/s]
120000lines [01:05, 1831.94lines/s]
7600lines [00:04, 1680.21lines/s]


In [6]:
# Import the neural network package from PyTorch 
import torch.nn as nn

# We also want to the functional module which contains all the functions in the torch.nn library.
import torch.nn.functional as F

# Creating a class called TextSentiment that takes in the neural network module from Pytorch.
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        # The EmbeddingBag module finds the sum or the mean of all the embeddings.
        # An embedding allows you to map low-dimensional real vectors that can represent each words to other words that are similar. 
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        # This does a Linear Transformation which allows the model to learn the weights between the embeddings and maps it to an output class
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        # intialization range set at 0.5
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    # This is the forward propagation which essentially feeds input data into each layer of the model 
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [7]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUM_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUM_CLASS)

print("The vocabulary size is: ", VOCAB_SIZE)
print("The number of classes is: ", NUM_CLASS)

The vocabulary size is:  1308844
The number of classes is:  4


In [1]:
def generate_batch(batch):
    # A tensor is used to represent n-Dimensions of features. It looks like a matrix but it's not a matrix, a matrix is simply used to visualize a tensor. 
    # The first part of the entry is the class label (e.g. what type of news article).
    label = torch.tensor([entry[0] for entry in batch])
    # The second part of the entry is the text. 
    text = [entry[1] for entry in batch]
    # These are the offset values in storage which indicates where the tensors start from.
    offsets = [0] + [len(entry) for entry in text]
    
    # torch.Tensor.cumsum returns a cumulative sum of all the elements in the dimension.
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    
    # torch.cat concatenates whatever you give it together. 
    text = torch.cat(text)
    return text, offsets, label